In [6]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer #categoriccal data into vectors
from sklearn.metrics.pairwise import cosine_similarity #compare genrs with one another
from ipywidgets import * #interactive widgets

In [7]:
#imprt the csv files
movies = pd.read_csv('datasets/movies.csv')
ratings = pd.read_csv('datasets/ratings.csv')


In [8]:
ratings.head()


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [9]:
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
#null values
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [11]:
movies.shape

(10329, 3)

In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [13]:
ratings.shape

(105339, 4)

In [14]:
#so no null values

In [15]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [16]:
# max rating = 5 min rating = 0.5

In [17]:
#different genres available

In [18]:
movies['genres'] = movies['genres'].str.split('|')

In [19]:
movies2 = movies.explode('genres') #to split each val in list into rows

In [20]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [21]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [22]:
movies2['genres'].nunique()


20

In [23]:
#we want to remove"no genres listed"

In [24]:
movies2 = movies2[movies2['genres'] != '(no genres listed)']

In [25]:
movies2['genres'].nunique()

19

In [26]:
movies2['genres'].unique()


array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [27]:
#group the dataframe based upon genres and we will get the average rating for each genre
#merge movies2 and ratng df
merged_data = pd.merge(ratings, movies2, on=['movieId'], how='inner')

In [28]:
merged_data.groupby('genres').groups

{'Action': [1941, 1944, 1947, 1950, 1953, 1956, 1959, 1962, 1965, 1968, 1971, 1974, 1977, 1980, 1983, 1986, 1989, 1992, 1995, 1998, 2001, 2004, 2007, 2010, 2013, 2016, 2019, 2022, 2025, 2028, 2031, 2034, 2037, 2040, 2043, 2046, 2049, 2052, 2055, 2058, 2061, 2064, 2067, 2070, 2073, 2076, 2079, 2082, 2085, 2088, 2091, 2094, 2097, 2100, 2103, 2106, 2109, 2112, 2115, 2118, 2121, 2124, 2127, 2130, 2133, 2136, 2139, 2142, 2145, 2148, 2151, 2154, 2157, 2160, 2163, 2166, 2169, 2172, 2175, 2178, 2181, 2184, 2187, 2190, 2193, 2196, 2199, 2202, 2205, 2208, 2211, 2214, 2217, 2220, 2223, 2226, 2229, 2232, 2235, 2238, ...], 'Adventure': [2685, 2688, 2691, 2694, 2697, 2700, 2703, 2706, 2709, 2712, 2715, 2718, 2721, 2724, 2727, 2730, 2733, 2736, 2739, 2742, 2745, 2748, 2751, 2754, 2757, 2760, 2763, 2766, 2769, 2772, 2775, 2778, 2781, 2784, 2787, 2790, 2793, 2796, 2799, 2802, 2805, 2808, 2811, 2814, 2817, 2820, 2823, 2826, 2829, 2832, 2835, 2838, 2841, 2844, 2847, 2850, 2853, 2856, 2859, 2862, 2865, 28

In [29]:
merged_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,9,16,4.0,842686699,Casino (1995),Crime
3,9,16,4.0,842686699,Casino (1995),Drama
4,12,16,1.5,1144396284,Casino (1995),Crime


In [30]:
merged_data.shape

(281897, 6)

In [31]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [32]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [33]:
# merged_data.groupby('genres').agg({"title":"nunique", 'rating':['mean','size']}).rename(columns={"title":"unique_movie_counts", "rating":"average_mean_rating"})

In [34]:
popularity = merged_data.groupby(['genres', 'title']).agg({'rating':['mean','size']}).rename(columns={ "rating":"average_mean_rating"}).reset_index()

In [35]:
popularity.columns=["Genres", "Title", "Average Ratings", "Number of Ratings"]

In [36]:
popularity

,Genres,Title,Average Ratings,Number of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [37]:
#popularity recommender system
popularity[popularity['Genres'] == "Action"]

,Genres,Title,Average Ratings,Number of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
1731,Action,Zu: Warriors from the Magic Mountain (Xin shu ...,4.000000,1
1732,Action,Zulu (1964),4.142857,7
1733,Action,eXistenZ (1999),3.660714,28
1734,Action,xXx (2002),2.958333,24


In [38]:
#genres = Action, threshold = 50, top =7
popularity[(popularity['Genres'] == "Action") & (popularity['Number of Ratings'] > 50)]


,Genres,Title,Average Ratings,Number of Ratings
13,Action,28 Days Later (2002),3.932836,67
20,Action,300 (2007),3.577922,77
43,Action,"Abyss, The (1989)",3.681319,91
62,Action,Air Force One (1997),3.439024,82
71,Action,Aliens (1986),4.146497,157
...,...,...,...,...
1638,Action,V for Vendetta (2006),4.015957,94
1676,Action,Waterworld (1995),2.934959,123
1692,Action,Wild Wild West (1999),2.164179,67
1707,Action,X-Men (2000),3.557692,130


In [39]:
popularity[(popularity['Genres'] == "Action") & (popularity['Number of Ratings'] > 50)].sort_values(by="Average Ratings", ascending=False).head(7)


,Genres,Title,Average Ratings,Number of Ratings
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


In [40]:
#popularity recommender system
def TopNPopularMovies(genre, threshold, topN):
    popularity = merged_data.groupby(['genres', 'title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres", "Title", "Average Ratings", "Number of Ratings"]
    
    #filter the dataon
    topNRecommendations = popularity[(popularity['Genres'] == genre) & (popularity['Number of Ratings'] > threshold)].sort_values(by="Average Ratings", ascending=False).head(topN)
    
    #ouput
    topNRecommendations['Sno.'] = range(1, len(topNRecommendations)+1)
    topNRecommendations.index = range(0, len(topNRecommendations))
    topNRecommendations.columns = ['Genres', 'Movie Title', 'Average Movie Ratings', 'Number of Reviews', 'Sno.']
    return topNRecommendations[['Sno.', 'Movie Title', 'Average Movie Ratings', 'Number of Reviews']]
    

In [41]:
#test cases 1
genre = 'Adventure'
threshold = 50
topN = 20
TopNPopularMovies(genre = genre, threshold = threshold, topN = topN)

,Sno.,Movie Title,Average Movie Ratings,Number of Reviews
0,1,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1,2,Monty Python and the Holy Grail (1975),4.301948,154
2,3,North by Northwest (1959),4.273973,73
3,4,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.236111,72
4,5,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
5,6,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
6,7,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
7,8,Star Wars: Episode IV - A New Hope (1977),4.188645,273
8,9,Lawrence of Arabia (1962),4.166667,60
9,10,"Princess Bride, The (1987)",4.163743,171


In [42]:
#content based recommendation system

In [43]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [44]:
movies3 = movies2.groupby('title').agg({'genres': lambda x:" ".join(list(x))}).reset_index()

In [45]:
movies3.head()

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy


In [46]:
#instance of tfidfvectorizer >> used ti apply fn upon genres and generate their vectors
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), stop_words="english")
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [47]:
tf_matrix = tf.fit_transform(movies3['genres'])

In [48]:
cosine_sim = cosine_similarity(tf_matrix, tf_matrix)

In [49]:
# movies3['title']

In [50]:
# indices = pd.Series(movies3.index, index=movies3['title'])

In [51]:
# idx = indices['xXx (2002)']

In [52]:
# scores = list(enumerate(cosine_sim[idx])) #similarity of xXx with all the other movies

In [53]:
# matched = sorted(scores, key = lambda x:x[1], reverse = True)[:5]
# matched = [i[0] for i in matched]
# matched

In [54]:
# movies3.iloc[matched] # five moveis related to xXx

In [55]:
#final function

In [56]:
def recommendationGenre(movie_df, similarity_matrix, movie_title, topN):
    indices = pd.Series(movies3.index, index=movies3['title'])
    idx = indices[movie_title] #target movie
    
    #similarity
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key = lambda x:x[1], reverse = True)[1:topN+2]
    
    #extract matching movies
    matched = [i[0] for i in scores]
    matching_df = movie_df.iloc[matched]
    
    #filter out the target movie
    matching_df = matching_df[matching_df['title'] != movie_title]
    
    #output
    matching_df.rename(columns={'title': 'Movie Title'}, inplace = True)
    matching_df['Sno.'] = range(1, len(matching_df)+1)
    matching_df.index = range(0, len(matching_df))
    
    return matching_df[['Sno.', 'Movie Title']].head(topN)

In [57]:
#test case
recommendationGenre(movie_df=movies3, similarity_matrix=cosine_sim, movie_title='Turbo (2013)', topN=20)

,Sno.,Movie Title
0,1,Antz (1998)
1,2,Asterix and the Vikings (Astérix et les Viking...
2,3,"Boxtrolls, The (2014)"
3,4,DuckTales: The Movie - Treasure of the Lost La...
4,5,"Emperor's New Groove, The (2000)"
5,6,"Monsters, Inc. (2001)"
6,7,Shrek the Third (2007)
7,8,"Tale of Despereaux, The (2008)"
8,9,Toy Story (1995)
9,10,Toy Story 2 (1999)


In [58]:
#interactive widgets

In [59]:
#popularity
#inputs
genres = Dropdown(options = set(movies2['genres']), description = 'Genres',style={"description_width":"initial"})
num_reviews = IntText(description="Minimum Reviews", style={"description_width":"initial"})
num_recommendations_1 = IntText(description = "Number of Recommnedations", style={"description_width":"initial"})

#tabs
b1 = Button(description = "Recommend ME", style={"description_width":"initial"})
h1 = HBox([num_reviews, num_recommendations_1])
popularity_tab = VBox([genres, h1, b1])

#content based widgets

title = Textarea(description="Movie Title", style={"description_width":"initial"})
num_recommendations_2 = IntText(description = "Number of Recommendations", style={"description_width":"initial"})

#tabs
h2 = HBox([title, num_recommendations_2])
b2 = Button(description = "Recommend ME", style={"description_width":"initial"})
content_tab = VBox([h2, b2])

#creating final tabs
tabs = [popularity_tab, content_tab]
wid = widgets.Tab(tabs)
# wid

#set titles to the tabs
names = ['Popularity Based Recommendations', 'Content Based Recommendations']
[wid.set_title(i, title) for i, title in enumerate(names)]

display(wid)

In [60]:
#popularity
def b1_clicked(b):
    global output
    result = TopNPopularMovies(genre = genres.value, threshold = num_reviews.value, topN = num_recommendations_1.value)
    output = result
b1.on_click(b1_clicked)

#content
def b2_clicked(b):
    global output
    result = recommendationGenre(movie_df=movies3, similarity_matrix=cosine_sim, movie_title= title.value, topN= num_recommendations_2.value)
    output = result
b2.on_click(b2_clicked)
    

In [61]:
display(wid)

In [62]:
output

NameError: name 'output' is not defined